In [1]:
using GridapTopOpt, Gridap, Gridap.TensorValues

In [2]:
const E = 1      #1
const ν = 0.3    #0.35#0.3
const G = E/(2*(1+ν))
const l = 4.0    #2.0##0.5#1.0#1.25 # Modified
const N = 0.3    #0.5#0#0.7

# const λₘₐₜ = 2*G*ν/(1 -2*ν)
# const κₘₐₜ = 2*G*N^2/(1-N^2)
# const μₘₐₜ = G*(1-2*(N^2))/(1-N^2)
# const γₘₐₜ = 4*G*l^2

const λₘₐₜ = E*ν/(1-ν^2)  # Modified λ for plane stress (critical change!)   #2*G*ν/(1 -2*ν)
const κₘₐₜ = 2*G*N^2/(1-N^2)
const μₘₐₜ = G*(1-2*(N^2))/(1-N^2)
const γₘₐₜ = 4*G*l^2

24.615384615384613

In [3]:
const Height = 10
const Length = 3*Height

30

In [4]:
# FE parameters
order = 1                                                        # Finite element order
dom = (0,Length,0,Height)                                            # Bounding domain
nx,ny = (300,100)#(113,57)#(150,75)
el_size = (nx,ny)#(300,100)                                              # Mesh partition size
f_Γ_N(x) = (x[2] ≈ Height) &&                                       # Γ_N indicator function
    (0.0 <= x[1] <= 5*Length/nx)
f_Γ_D1(x) = (x[1] ≈ 0.0)
f_Γ_D2(x) = (x[1] ≈ Length) && (x[2] ≈ 0.0)

f_Γ_D2 (generic function with 1 method)

In [5]:
# FD parameters
γ = 0.1                                                          # HJ eqn time step coeff
γ_reinit = 0.5#0.1                                                   # Reinit. eqn time step coeff
max_steps = 45#20#25#floor(Int,order*maximum(el_size)/10)                 # Max steps for advection
#tol = 1/(5order^2)/minimum(el_size)#0.00007#1/(5order^2)/minimum(el_size)                             
tol =1e-6#1/(5order^2*Length^2)#/minimum(el_size)                              # Reinitialisation tolerance

1.0e-6

In [6]:
# Problem parameters
g = VectorValue(0,-1)                                            # Heat flow in
vf = 0.3                                                         # Volume fraction constraint
lsf_func = initial_lsf((12/Length),0.2)#initial_lsf((8/Length),0.2)                                    # Initial level set function
iter_mod = 10                                                    # VTK Output modulo                        
path = "./Result_tol&Max_stepMod/results_315times105/N_$N/ElasticMicropolar_l_$l/$Length,$Height/comp_serial/E_1/"      # Output path
mkpath(path)                                                     # Create path

"./Result_tol&Max_stepMod/results_315times105/N_0.3/ElasticMicropolar_l_4.0/30,10/comp_serial/E_1"

In [7]:
function σ_Bmod(ε)
    σM = ((λₘₐₜ)*tr(ε)*one(ε) + (2*μₘₐₜ + κₘₐₜ)*(ε))
    return σM
end

E_Matrx = TensorValue(0,1,-1,0)

function ε_Skw(∇,θ)
    ∇ᵀ = transpose(∇)
    w = (0.5*(∇ᵀ - ∇)) - (E_Matrx*θ)
    return w
end

function σ_Cmod(ϵ_skew)
    σM = κₘₐₜ*ϵ_skew
    return σM
end

function M_mod(∇)
    M = γₘₐₜ*∇
    return M
end

function Skw(u,θ)
    ∇ᵀ = transpose(∇(u))
    w = (0.5*(∇ᵀ - ∇(u)) - (E_Matrx*θ))
    return w
end

function ElasEnergy(ε_in)
    ElasEner = 0.5*(λₘₐₜ)*(tr(ε_in) * tr(ε_in)) + (μₘₐₜ + 0.5*κₘₐₜ)*(ε_in ⊙ ε_in)
    TotElasEner = sum(∫(ElasEner)*dΩ)
    return TotElasEner
end

function RotationEnergy(∇_th) #∇_th = ∇(θ)
    RotEnergy = 0.5*γₘₐₜ*(∇_th ⋅ ∇_th)
    TotRotEnergy = sum(∫(RotEnergy)*dΩ)
    return TotRotEnergy
end

function CouplingEnergy(∇_uh,θ)
    ∇_uhᵀ = transpose(∇_uh)
    ϵ_skew = 0.5*(∇_uhᵀ - ∇_uh) - E_Matrx*θ
    coupEnergy = 0.5*(κₘₐₜ*(ϵ_skew ⊙ ϵ_skew))
    TotcoupEnergy = sum(∫(coupEnergy)*dΩ)
    return TotcoupEnergy
end

CouplingEnergy (generic function with 1 method)

In [8]:
# Model
model = CartesianDiscreteModel(dom,el_size);
update_labels!(1,model,f_Γ_D1,"Gamma_D1")
update_labels!(2,model,f_Γ_D2,"Gamma_D2")
update_labels!(3,model,f_Γ_N,"Gamma_N")

In [9]:
writevtk(model,path*"Half_MBBExampleI")

3-element Vector{Vector{String}}:
 ["./Result_tol&Max_stepMod/results_315times105/N_0.3/ElasticMicropolar_l_4.0/30,10/comp_serial/E_1/Half_MBBExampleI_0.vtu"]
 ["./Result_tol&Max_stepMod/results_315times105/N_0.3/ElasticMicropolar_l_4.0/30,10/comp_serial/E_1/Half_MBBExampleI_1.vtu"]
 ["./Result_tol&Max_stepMod/results_315times105/N_0.3/ElasticMicropolar_l_4.0/30,10/comp_serial/E_1/Half_MBBExampleI_2.vtu"]

In [10]:
# Triangulation and measures
Ω = Triangulation(model)
Γ_N = BoundaryTriangulation(model,tags="Gamma_N")
dΩ = Measure(Ω,2*order)
dΓ_N = Measure(Γ_N,2*order)
vol_D = sum(∫(1)dΩ)

299.9999999999999

In [11]:
## Spaces
reffe = ReferenceFE(lagrangian,VectorValue{2,Float64},order)
reffe_scalar = ReferenceFE(lagrangian,Float64,order)
V = TestFESpace(model,reffe;conformity=:H1,dirichlet_tags=["Gamma_D1","Gamma_D2"],dirichlet_masks = [(true,false),(false,true)])
U = TrialFESpace(V,[VectorValue(0.0,0.0),VectorValue(0.0,0.0)])   ## Dispalcement Space
Q = TestFESpace(model,reffe_scalar;conformity=:H1)#,dirichlet_tags=["Gamma_D"],dirichlet_masks = [(true)])
P = TrialFESpace(Q)#,[0.0]) ## Theta(Rotation) Space

UnconstrainedFESpace()

In [12]:
UP = MultiFieldFESpace([U,P])
VQ = MultiFieldFESpace([V,Q])

MultiFieldFESpace()

In [13]:
V_φ = TestFESpace(model,reffe_scalar)
V_reg = TestFESpace(model,reffe_scalar)
U_reg = TrialFESpace(V_reg)

UnconstrainedFESpace()

In [14]:
# Level set and interpolator
φh = interpolate(lsf_func,V_φ)
interp = SmoothErsatzMaterialInterpolation(η = (2)*maximum(get_el_Δ(model)))    # η = 2 ×  maximum side length of an element.
I,H,DH,ρ = interp.I,interp.H,interp.DH,interp.ρ

(GridapTopOpt.var"#I#298"{GridapTopOpt.var"#H#296"{Vector{Float64}}, Vector{Float64}}(GridapTopOpt.var"#H#296"{Vector{Float64}}([0.2]), [0.001]), GridapTopOpt.var"#H#296"{Vector{Float64}}([0.2]), GridapTopOpt.var"#DH#297"{Vector{Float64}}([0.2]), GridapTopOpt.var"#ρ#299"{GridapTopOpt.var"#H#296"{Vector{Float64}}}(GridapTopOpt.var"#H#296"{Vector{Float64}}([0.2])))

In [15]:
writevtk(Ω,path*"initial_lsfHalfMBBI",cellfields=["phi"=>φh,
  "ρ(phi)"=>(ρ ∘ φh),"|nabla(phi)|"=>(norm ∘ ∇(φh))])

(["./Result_tol&Max_stepMod/results_315times105/N_0.3/ElasticMicropolar_l_4.0/30,10/comp_serial/E_1/initial_lsfHalfMBBI.vtu"],)

In [16]:
a((u,θ),(w,v),φ) = ∫((I ∘ φ)*((ε(w) ⊙ (σ_Bmod∘(ε(u))) ) + ((Skw(w,v)) ⊙ (σ_Cmod∘(ε_Skw∘(∇(u),θ)))) + ((∇(v))⋅ (M_mod∘(∇(θ)))) - ((v*((E_Matrx) ⊙ (σ_Cmod∘(ε_Skw∘(∇(u),θ))))) )))dΩ;

lm((w,v),φ) = ∫(w·g)dΓ_N


lm (generic function with 1 method)

In [17]:
state_map = RepeatingAffineFEStateMap(1, a, [lm],UP,VQ,V_φ)#,U_reg,φh,dΩ,dΓ_N)

RepeatingAffineFEStateMap

In [18]:
#stencil = HamiltonJacobiEvolution(FirstOrderStencil(2,Float64),model,V_φ,tol,max_steps)
evo = FiniteDifferenceEvolver(FirstOrderStencil(2,Float64),model,V_φ;max_steps)
  reinit = FiniteDifferenceReinitialiser(FirstOrderStencil(2,Float64),model,V_φ;tol,γ_reinit)
  ls_evo = LevelSetEvolution(evo,reinit)

LevelSetEvolution{FiniteDifferenceEvolver{1}, FiniteDifferenceReinitialiser{1}}(FiniteDifferenceEvolver{1}(FirstOrderStencil{2, Float64}(), CartesianDiscreteModel(), UnconstrainedFESpace(), [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  30392, 30393, 30394, 30395, 30396, 30397, 30398, 30399, 30400, 30401], (isperiodic = (false, false), Δ = (0.1, 0.1), ndof = (301, 101), max_steps = 45, correct_ls = false)), FiniteDifferenceReinitialiser{1}(FirstOrderStencil{2, Float64}(), CartesianDiscreteModel(), UnconstrainedFESpace(), [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  30392, 30393, 30394, 30395, 30396, 30397, 30398, 30399, 30400, 30401], (isperiodic = (false, false), Δ = (0.1, 0.1), ndof = (301, 101), max_steps = 2000, tol = 1.0e-6, γ_reinit = 0.5, correct_ls = false)))

In [19]:
function C_plane_stress(E,ν)
    λ = E*ν/(1-ν^2)  # Plane stress λ
    μ = E/(2*(1+ν))
    return SymFourthOrderTensorValue{2}(
        λ+2μ, 0.0, λ,
        0.0, μ, 0.0,
        λ, 0.0, λ+2μ
    )
end
C2 = C_plane_stress(1,0.3)

# function C_plane_strain(E, ν)
#     λ = E * ν / ((1 + ν) * (1 - 2ν))  # Plane strain λ
#     μ = E / (2 * (1 + ν))             # Shear modulus
#     return SymFourthOrderTensorValue{2}(
#         λ + 2μ, 0.0, λ,
#         0.0, μ, 0.0,
#         λ, 0.0, λ + 2μ
#     )
# end

# C2 = C_plane_strain(1, 0.3)

SymFourthOrderTensorValue{2, Float64, 9}(1.0989010989010988, 0.0, 0.32967032967032966, 0.0, 0.3846153846153846, 0.0, 0.32967032967032966, 0.0, 1.0989010989010988)

In [20]:
function Cᴴ(r,s,uϕ,φ,dΩ,dΓ_N)
    u_s = uϕ[2s-1]; θ_s = uϕ[2s];
    ∫((u_s)·g)dΓ_N # ∫((I ∘ φ)*(C2 ⊙ (ε(u_s)) ⊙ (ε(u_s))))dΩ #
end



J(uϕ,φ) = 1*Cᴴ(1,1,uϕ,φ,dΩ,dΓ_N)
C1(uϕ,φ) = ∫(((ρ ∘ φ) - vf)/vol_D)dΩ;

In [22]:
DC1(q,uϕ,φ) = ∫(-1/vol_D*q*(DH ∘ φ)*(norm ∘ ∇(φ)))dΩ

DC1 (generic function with 1 method)

In [23]:
pcfs = PDEConstrainedFunctionals(J,[C1],state_map,analytic_dJ=nothing,analytic_dC=[DC1])

PDEConstrainedFunctionals:
    num_constraints: 1

In [24]:
α = 4max_steps*γ*maximum(get_el_Δ(model))
a_hilb(p,q) = ∫(α^2*∇(p)⋅∇(q) + p*q)dΩ;
vel_ext = VelocityExtension(a_hilb,U_reg,V_reg)

VelocityExtension

In [25]:
# ## Optimiser
optimiser = AugmentedLagrangian(pcfs,ls_evo,vel_ext,φh;γ,verbose=true,constraint_names=[:Vol])

AugmentedLagrangian

In [26]:
for (it,uh,φh) in optimiser
    uv, Tv = uh
    data = ["φ"=>φh,"H(φ)"=>(H ∘ φh),"|∇(φ)|"=>(norm ∘ ∇(φh)),"uv"=>uv,"Tv"=>Tv,"ρ(φ)"=>(ρ ∘ φh)]
    iszero(it % iter_mod) && writevtk(Ω,path*"out$it",cellfields= data) 
    write_history(path*"/historymodified$tol,max_steps-$max_steps.txt",optimiser.history)
end

Iteration:   0 | L=5.1189e+01, J=4.9645e+01, Vol=3.8704e-01, γ=1.0000e-01, λ1=0.0000e+00, Λ1=2.0618e+01
Iteration:   1 | L=3.9032e+01, J=3.7033e+01, Vol=4.4037e-01, γ=1.0000e-01, λ1=-9.0795e+00, Λ1=2.0618e+01
Iteration:   2 | L=3.7190e+01, J=3.0264e+01, Vol=4.9010e-01, γ=1.0000e-01, λ1=-1.9184e+01, Λ1=2.0618e+01
Iteration:   3 | L=4.0123e+01, J=2.7251e+01, Vol=5.2360e-01, γ=1.0000e-01, λ1=-2.9980e+01, Λ1=2.0618e+01
Iteration:   4 | L=4.4333e+01, J=2.6363e+01, Vol=5.0998e-01, γ=1.0000e-01, λ1=-4.0495e+01, Λ1=2.0618e+01
Iteration:   5 | L=4.8248e+01, J=2.8066e+01, Vol=4.4742e-01, γ=1.0000e-01, λ1=-4.9720e+01, Λ1=2.2680e+01
Iteration:   6 | L=5.0606e+01, J=3.0042e+01, Vol=3.8057e-01, γ=1.0000e-01, λ1=-5.8351e+01, Λ1=2.2680e+01
Iteration:   7 | L=5.1730e+01, J=3.1360e+01, Vol=3.2817e-01, γ=1.0000e-01, λ1=-6.5794e+01, Λ1=2.2680e+01
Iteration:   8 | L=5.5022e+01, J=3.5163e+01, Vol=2.8758e-01, γ=1.0000e-01, λ1=-7.2316e+01, Λ1=2.2680e+01
Iteration:   9 | L=5.4883e+01, J=3.2751e+01, Vol=2.9262e

In [ ]:
function ElasEnergy(ε_in)
    ElasEner = 0.5*(λₘₐₜ)*(tr(ε_in))^2 + (μₘₐₜ + 0.5*κₘₐₜ)*(ε_in ⊙ ε_in)
    TotElasEner = sum(∫(ElasEner)*dΩ)
    return TotElasEner
end

function RotationEnergy(∇_th) #∇_th = ∇(θ)
    RotEnergy = 0.5*γₘₐₜ*(∇_th ⋅ ∇_th)
    TotRotEnergy = sum(∫(RotEnergy)*dΩ)
    return TotRotEnergy
end

function CouplingEnergy(∇_uh,θ)
    ∇_uhᵀ = transpose(∇_uh)
    ϵ_skew = 0.5*(∇_uhᵀ - ∇_uh) - E_Matrx*θ
    coupEnergy = 0.5*(κₘₐₜ*(ϵ_skew ⊙ ϵ_skew))
    TotcoupEnergy = sum(∫(coupEnergy)*dΩ)
    return TotcoupEnergy
end

In [27]:
it = get_history(optimiser).niter; uh = get_state(pcfs);
uv, Tv = uh
WElas = ElasEnergy(ε(uv))
WRot = RotationEnergy(∇(Tv))
WElas = CouplingEnergy(∇(uv),Tv)

writevtk(Ω,path*"out$it,max_step-$max_steps, tol-$tol",cellfields=["φ"=>φh,"H(φ)"=>(H ∘ φh),"|∇(φ)|"=>(norm ∘ ∇(φh)),"uv"=>uv,"Tv"=>Tv])

┌ Warning: detected extension '.0e-6' does not correspond to dataset type.
│ Appending '.vtu' to filename.
└ @ WriteVTK ~/.julia/packages/WriteVTK/Be3qm/src/WriteVTK.jl:162


(["./Result_tol&Max_stepMod/results_315times105/N_0.3/ElasticMicropolar_l_4.0/30,10/comp_serial/E_1/out84,max_step-45, tol-1.0e-6.vtu"],)

In [23]:
WElas = ElasEnergy(ε(uv))
WRot = RotationEnergy(∇(Tv))
WElas = CouplingEnergy(∇(uv),Tv)

0.0